In [0]:
%sql
create table if not exists dev.demo_db.invoices_raw_2
(
  InvoiceNo int,
  StockCode string,
  Description string,
  Quantity int,
  InvoiceDate timestamp,
  UnitPrice double,
  CustomerID string
)

#### Ingest Into Using Spark Streaming API

In [0]:
def ingest():
    invoice_schema = """InvoiceNo int, StockCode string, Description string, Quantity int, InvoiceDate timestamp, UnitPrice double, CustomerID string"""

    source_df = (spark.readStream
                 .format("csv")
                 .option("header", "true")
                 .schema(invoice_schema)
                 .load("/Volumes/dev/demo_db/landing")
                 )
    
    write_query = (source_df.writeStream
                  .format("delta")
                  .option("checkpointLocation", "/Volumes/dev/demo_db/landing/checkpoint/invoice_final")
                  .outputMode("append")
                  .trigger(availableNow=True)
                  .toTable("dev.demo_db.invoices_raw_2")
                  )
    
ingest()

In [0]:
%sql
select * from dev.demo_db.invoices_raw_2

In [0]:
%fs cp /Volumes/dev/demo_db/landing/invoices/invoices_2021.csv /Volumes/dev/demo_db/landing

In [0]:
%fs ls /Volumes/dev/demo_db/landing

#### Alter table to evolve the schema

In [0]:
%sql
alter table dev.demo_db.invoices_raw_2 add columns (Country String)

In [0]:
def ingest():
    invoice_schema = """InvoiceNo int, StockCode string, Description string, Quantity int, InvoiceDate timestamp, UnitPrice double, CustomerID string, Country String"""

    source_df = (spark.readStream
                 .format("csv")
                 .option("header", "true")
                 .schema(invoice_schema)
                 .load("/Volumes/dev/demo_db/landing")
                 )
    
    write_query = (source_df.writeStream
                  .format("delta")
                  .option("checkpointLocation", "/Volumes/dev/demo_db/landing/checkpoint/invoice_final")
                  .outputMode("append")
                  .trigger(availableNow=True)
                  .toTable("dev.demo_db.invoices_raw_2")
                  )
    
ingest()

In [0]:
%sql
select * from dev.demo_db.invoices_raw_2